In [1]:
from keras import losses, metrics
from keras.models import Sequential
from keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, sys, pickle
import pandas as pd
import random

# GPU setting

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

# 1. Loading input

In [3]:
import pandas as pd
import numpy as np

BATCH_SIZE = 1024
MAX_CASES = 2000
SEGLEN_IN_SEC = 20
SRATE = 100
LEN_INPUT = 20
OVERLAP = 10
LEN_PER_PRE = 60
LEN_PER_POST = 60


print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = f"../DL_model/dataset/ne{LEN_PER_PRE}s-e{LEN_PER_POST}s-len{LEN_INPUT}-{OVERLAP}/"
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
x_val = np.load(input_path+'x_val.npz', allow_pickle=True)['arr_0']

tss_train = np.load(input_path+'tss_train.npz')['arr_0']
tss_test = np.load(input_path+'tss_test.npz')['arr_0']
tss_val = np.load(input_path+'tss_val.npz')['arr_0']
cisa_train = np.load(input_path+'cisa_train.npz')['arr_0']
cisa_test = np.load(input_path+'cisa_test.npz')['arr_0']
cisa_val = np.load(input_path+'cisa_val.npz')['arr_0']

gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
gender_test = np.load(input_path+'gender_test.npz', allow_pickle=True)['arr_0']
gender_val = np.load(input_path+'gender_val.npz', allow_pickle=True)['arr_0']

age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
age_test = np.load(input_path+'age_test.npz', allow_pickle=True)['arr_0']
age_val = np.load(input_path+'age_val.npz', allow_pickle=True)['arr_0']


print('done', flush=True)


print('x_train shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('x_val.shape:', x_val.shape)

loading train...done
x_train shape: (9135, 2000, 2)
x_test.shape: (1159, 2000, 2)
x_val.shape: (1023, 2000, 2)


## Input, output settings

In [4]:
# input mode
mode_in = 'PPG+ECG'

if mode_in == 'PPG':
    x_train = x_train[:,:,0:1]
    x_test = x_test[:,:,0:1]
    x_val = x_val[:,:,0:1]
elif mode_in == 'ECG':
    x_train = x_train[:,:,1:2]
    x_test = x_test[:,:,1:2]
    x_val = x_val[:,:,1:2]

In [5]:
mode = 'TSS'

if mode == 'TSS':
    # tss를 output으로 할 경우
    y_train = tss_train
    y_val = tss_val
    y_test = tss_test

elif mode == 'CISA':
    # cisa를 output으로 할 경우
    y_train = cisa_train
    y_val = cisa_val
    y_test = cisa_test

## Sample weights

In [6]:
# 2 class에 대한 sample weight
y_train_bin = y_train > 0
y_val_bin = y_val > 0
y_test_bin = y_test > 0

y_train_bin2 = y_train > 0.2
y_val_bin2 = y_val > 0.2
y_test_bin2 = y_test > 0.2


train_w_samp2 = np.ones(shape=(len(y_train),))
train_w_samp2[y_train_bin>0]= len(y_train) / np.sum(y_train_bin)
train_w_samp2[y_train_bin==0]= len(y_train) / np.sum(~y_train_bin)

train_w_samp2_2 = np.ones(shape=(len(y_train),))
train_w_samp2_2[y_train_bin2>0]= len(y_train) / np.sum(y_train_bin2)
train_w_samp2_2[y_train_bin2==0]= len(y_train) / np.sum(~y_train_bin2)

print('<train set>')
print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_train) / np.sum(~y_train_bin), len(y_train) / np.sum(y_train_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_train) / np.sum(~y_train_bin2), len(y_train) / np.sum(y_train_bin2)))

# 2 class에 대한 sample weight
val_w_samp2 = np.ones(shape=(len(y_val),))
val_w_samp2[y_val_bin>0]= len(y_val) / np.sum(y_val_bin)
val_w_samp2[y_val_bin==0]= len(y_val) / np.sum(~y_val_bin)

val_w_samp2_2 = np.ones(shape=(len(y_val),))
val_w_samp2_2[y_val_bin2>0]= len(y_val) / np.sum(y_val_bin2)
val_w_samp2_2[y_val_bin2==0]= len(y_val) / np.sum(~y_val_bin2)

print('<val set>')
print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_val) / np.sum(~y_val_bin), len(y_val) / np.sum(y_val_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_val) / np.sum(~y_val_bin2), len(y_val) / np.sum(y_val_bin2)))

# 2 class에 대한 sample weight
test_w_samp2 = np.ones(shape=(len(y_test),))
test_w_samp2[y_test_bin>0]= len(y_test) / np.sum(y_test_bin)
test_w_samp2[y_test_bin==0]= len(y_test) / np.sum(~y_test_bin)

test_w_samp2_2 = np.ones(shape=(len(y_test),))
test_w_samp2_2[y_test_bin2>0]= len(y_test) / np.sum(y_test_bin2)
test_w_samp2_2[y_test_bin2==0]= len(y_test) / np.sum(~y_test_bin2)

print('<test set>')
print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_test) / np.sum(~y_test_bin), len(y_test) / np.sum(y_test_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_test) / np.sum(~y_test_bin2), len(y_test) / np.sum(y_test_bin2)))

<train set>
sample weight for no pain: 1.42, moderate pain: 3.36
sample weight for no pain: 1.29, severe pain: 4.44
<val set>
sample weight for no pain: 1.52, moderate pain: 2.94
sample weight for no pain: 1.37, severe pain: 3.69
<test set>
sample weight for no pain: 1.43, moderate pain: 3.34
sample weight for no pain: 1.28, severe pain: 4.60


In [7]:
train_w_samp = train_w_samp2
val_w_samp = val_w_samp2
test_w_samp = test_w_samp2

# 2. Setting

In [8]:
# folder
nfold = 1  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/TSS/CNN_{mode_in}_4layers_Reg_{nfold}fold_test{ntest}_w_samp0_lr"

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: {mode_in} (pre-intubation 120~60s, post-intubation 60~120s), output: {mode}')
f.close()
    

# test_settings
test_settings_1, test_settings_2, test_settings_3 = [], [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.001

# hyperparamters pool
num_opts = [32, 64, 128, 256] # num of filters(kernel)
stride_opts = [1,1,1,1,1,2,2,2,2]
kernel_opts = range(3,9,2) # kernel size
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 8, 16, 32, 64, 128]
globalpool_opts = ['max','ave']
BATCH_SIZE = [512, 1024]


print('start making test settings...', end='', flush=True)
# test settings
for num_l1 in num_opts:
    for num_l2 in num_opts:
        for num_l3 in num_opts:
            for num_l4 in num_opts:
                for kernel_l1 in kernel_opts:
                    for kernel_l2 in kernel_opts:
                        for kernel_l3 in kernel_opts:
                            for kernel_l4 in kernel_opts:
                                test_settings_1.append([num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4])

for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for globalpool_opt in globalpool_opts:
                for batch_size in BATCH_SIZE:
                    for conv_double in [True, False]:
                        for learning_rate in [0.001, 0.002, 0.0005]:
                            test_settings_2.append([dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double, learning_rate])                                   

for stride_l1 in stride_opts:
    for stride_l2 in stride_opts:
        for stride_l3 in stride_opts:
            for stride_l4 in stride_opts:
                for stride_l5 in stride_opts:
                    for num_l5 in num_opts:
                        for kernel_l5 in kernel_opts:
                            test_settings_3.append([stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5])
                        
print('done')

start making test settings...done


# 3. Random Search

## Regression model

In [ ]:
from keras import metrics


# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
#test_roc, test_prc = [], []
test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    test_setting_3 = random.choice(test_settings_3)
        
        
    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double, learning_rate = test_setting_2
    stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5 = test_setting_3
    
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4])
    
    if n_conv==2:
        num_l3,kernel_l3,stride_l3 = 0,0,0
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
    
    if n_conv==3:
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
        
    if n_conv==4:
        num_l5,kernel_l5,stride_l5 = 0,0,0    
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch={batch_size},c1={num_l1},c2={num_l2},c3={num_l3},c4={num_l4},filt1={kernel_l1},filt2={kernel_l2},filt3={kernel_l3},filt4={kernel_l4},str1={stride_l1}, \
        str2={stride_l2},str3={stride_l3},str4={stride_l4}, conv_double={conv_double},globalpool={globalpool_opt},dropout={dropout_cnn},dnodes={dense_node},dropout={dropout_fc},lr={learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    
    with tf.device('/gpu:0'):
        # build a model
        model = Sequential()

        act='relu'

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # c2 layer
        if num_l1 == 512:
            model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if n_conv>2:
            if num_l2 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same'))
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same', activation=act))
            model.add(BatchNormalization())
            model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if n_conv>3:
            if num_l3 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same'))
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same', activation=act))
            model.add(BatchNormalization())
            model.add(MaxPooling1D(pool_size=pool_size))
            
        # c5 layer
        if n_conv>4:
            if num_l4 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                model.add(Conv1D(filters=num_l5, kernel_size=kernel_l5, strides=stride_l5, padding='same'))
            model.add(Conv1D(filters=num_l5, kernel_size=kernel_l5, strides=stride_l5,padding='same', activation=act))
            model.add(BatchNormalization())
            model.add(MaxPooling1D(pool_size=pool_size))            
            


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='tanh'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(1))


        # model 학습 설정
        try:
            model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["mean_absolute_error"])
            hist = model.fit(x_train, y_train, sample_weight = train_w_samp, validation_data=(x_val, y_val, val_w_samp), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except Exception as e:
            print(f'error: {e}')
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_mae.append(0)
            test_rmse.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            

    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test).flatten()

    # auroc 계산
    y_test_bin = y_test > 0
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_auc.append(roc_auc)
    
    # RMSE 계산
    model_err = metrics.RootMeanSquaredError() 
    model_err.update_state(y_test, y_pred)
    rmse_val = model_err.result().numpy()
    test_rmse.append(rmse_val)
    
    # MAE 계산
    model_err = metrics.MeanAbsoluteError()
    model_err.update_state(y_test, y_pred)
    mae_val = model_err.result().numpy()
    test_mae.append(mae_val)
    
    
    # acc 계산
    #acc_val = np.mean((y_pred*10>=5)==y_test_bin)
    #test_acc.append(acc_val)
    
    # rename
    os.rename(odir, f'{rootdir}/roc{roc_auc:.3f}_mae{mae_val:.3f}_rmse{rmse_val:.3f}_{odir_f}')

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()

max_idx = test_roc.index(max(test_auc))
print('\nBest Model mae:{:.4f}, info: {}'.format(test_mae(max_idx), random_settings(max_idx)))


random search 0/500


2022-02-13 02:21:25.083577: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-13 02:21:25.889977: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30267 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0
/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100


2022-02-13 02:21:31.004972: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-02-13 02:21:31.450540: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


18/18 [==============================] - ETA: 0s - loss: 1.3200 - mean_absolute_error: 0.5619
Epoch 00001: val_loss improved from inf to 0.68348, saving model to randomSearch/TSS/CNN_PPG+ECG_4layers_Reg_1fold_test500_w_samp0_lr/batch=512,c1=32,c2=256,c3=64,c4=0,filt1=3,filt2=5,filt3=5,filt4=0,str1=1,         str2=1,str3=2,str4=0, conv_double=True,globalpool=ave,dropout=0,dnodes=32,dropout=0.5,lr=0.001/weights.hdf5
18/18 [==============================] - 9s 260ms/step - loss: 1.3200 - mean_absolute_error: 0.5619 - val_loss: 0.6835 - val_mean_absolute_error: 0.5062
Epoch 2/100
18/18 [==============================] - ETA: 0s - loss: 0.3123 - mean_absolute_error: 0.2998
Epoch 00002: val_loss improved from 0.68348 to 0.17565, saving model to randomSearch/TSS/CNN_PPG+ECG_4layers_Reg_1fold_test500_w_samp0_lr/batch=512,c1=32,c2=256,c3=64,c4=0,filt1=3,filt2=5,filt3=5,filt4=0,str1=1,         str2=1,str3=2,str4=0, conv_double=True,globalpool=ave,dropout=0,dnodes=32,dropout=0.5,lr=0.001/weights.

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
18/18 [==============================] - ETA: 0s - loss: 1.0505 - mean_absolute_error: 0.5113
Epoch 00001: val_loss improved from inf to 0.05730, saving model to randomSearch/TSS/CNN_PPG+ECG_4layers_Reg_1fold_test500_w_samp0_lr/batch=512,c1=32,c2=32,c3=128,c4=0,filt1=7,filt2=5,filt3=7,filt4=0,str1=2,         str2=1,str3=1,str4=0, conv_double=False,globalpool=ave,dropout=0,dnodes=128,dropout=0.4,lr=0.002/weights.hdf5
18/18 [==============================] - 2s 67ms/step - loss: 1.0505 - mean_absolute_error: 0.5113 - val_loss: 0.0573 - val_mean_absolute_error: 0.1297
Epoch 2/100
16/18 [=========================>....] - ETA: 0s - loss: 0.2403 - mean_absolute_error: 0.2544
Epoch 00002: val_loss did not improve from 0.05730
18/18 [==============================] - 1s 30ms/step - loss: 0.2313 - mean_absolute_error: 0.2484 - val_loss: 0.0699 - val_mean_absolute_error: 0.1100
Epoch 3/100
16/18 [=========================>....] - ETA: 0s - loss: 0.1192 - mean_absolute_error: 0.1843
E

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
18/18 [==============================] - ETA: 0s - loss: 1.3700 - mean_absolute_error: 0.6574
Epoch 00001: val_loss improved from inf to 0.18297, saving model to randomSearch/TSS/CNN_PPG+ECG_4layers_Reg_1fold_test500_w_samp0_lr/batch=512,c1=64,c2=32,c3=64,c4=0,filt1=7,filt2=7,filt3=5,filt4=0,str1=1,         str2=2,str3=1,str4=0, conv_double=True,globalpool=max,dropout=0.5,dnodes=16,dropout=0.2,lr=0.002/weights.hdf5
18/18 [==============================] - 4s 119ms/step - loss: 1.3700 - mean_absolute_error: 0.6574 - val_loss: 0.1830 - val_mean_absolute_error: 0.2171
Epoch 2/100
 3/18 [====>.........................] - ETA: 0s - loss: 0.8509 - mean_absolute_error: 0.5139

## Binary classification

In [ ]:
'''
        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train_ecg, y_train_bin, sample_weight=train_w_samp3, validation_data=(x_val_ecg, y_val_bin, val_w_samp3), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test_ecg).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp3)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))

'''
